# Data Preprocessing

Now that I have collected the data, in this notebook, I proceed to clean it.

## Cleaning Player Box Score Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
player_box_df = pd.read_csv('./Data/player_box_scores.csv')

In [3]:
player_box_df.shape

(201805, 188)

In [4]:
player_box_df.head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PCT_PFD_RANK,PCT_PF_RANK,PCT_PTS,PCT_PTS_RANK,PCT_REB,PCT_REB_RANK,PCT_STL,PCT_STL_RANK,PCT_TOV,PCT_TOV_RANK
0,22015,1626162,Kelly Oubre Jr.,1610612764,WAS,Washington Wizards,21501221,2016-04-13,WAS vs. ATL,W,...,9233,14795,0.113,18901,0.103,18752,0.000,12313,0.167,13279
1,22015,202397,Ish Smith,1610612755,PHI,Philadelphia 76ers,21501222,2016-04-13,PHI @ CHI,L,...,17321,18077,0.127,17860,0.185,12326,0.000,12313,0.273,18957
2,22015,201166,Aaron Brooks,1610612741,CHI,Chicago Bulls,21501222,2016-04-13,CHI vs. PHI,W,...,18762,23985,0.000,23226,0.000,23167,0.000,12313,0.000,1
3,22015,203503,Tony Snell,1610612741,CHI,Chicago Bulls,21501222,2016-04-13,CHI vs. PHI,W,...,18745,6900,0.118,18513,0.135,16236,0.000,12313,0.067,9212
4,22015,203924,Jerami Grant,1610612755,PHI,Philadelphia 76ers,21501222,2016-04-13,PHI @ CHI,L,...,3572,10896,0.244,7484,0.240,8775,0.143,11077,0.111,10569


I have 188 columns in this dataframe. First, I want to explore what is going on in all of those columns.

In [5]:
column_list = player_box_df.columns.tolist()

In [6]:
column_list

['SEASON_ID',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'FANTASY_PTS',
 'VIDEO_AVAILABLE',
 'PLAYER_GAME_ID',
 'AST_PCT',
 'AST_PCT_RANK',
 'AST_RATIO',
 'AST_RATIO_RANK',
 'AST_TO',
 'AST_TO_RANK',
 'AVAILABLE_FLAG',
 'DEF_RATING',
 'DEF_RATING_RANK',
 'DREB_PCT',
 'DREB_PCT_RANK',
 'EFG_PCT',
 'EFG_PCT_RANK',
 'E_DEF_RATING',
 'E_DEF_RATING_RANK',
 'E_NET_RATING',
 'E_NET_RATING_RANK',
 'E_OFF_RATING',
 'E_OFF_RATING_RANK',
 'E_PACE',
 'E_PACE_RANK',
 'E_TOV_PCT',
 'E_TOV_PCT_RANK',
 'E_USG_PCT',
 'E_USG_PCT_RANK',
 'FGA_PG',
 'FGA_PG_RANK',
 'FGA_RANK',
 'FGM_PG',
 'FGM_PG_RANK',
 'FGM_RANK',
 'FG_PCT_RANK',
 'GP_RANK',
 'L_RANK',
 'MIN_RANK',
 'NET_RATING',
 'NET_RATING_RANK',
 'NICKNAME',
 'OFF_RATING',
 'OFF_RATING_RANK',


In [7]:
player_box_df['PCT_OREB_RANK'].head()

0    13060
1    13060
2    13060
3    13060
4     5356
Name: PCT_OREB_RANK, dtype: int64

Several of the statistics have a 'rank' feature attached to the feature itself. I don't think this is going to be helpful because it is a less precise version of the same statistic, simply showing where the player ranked among all players in that statistic for that season. I am going to drop all of the features that end in rank to get rid of those unnecessary features.

In [8]:
columns_to_drop = [col for col in player_box_df.columns if col.endswith('RANK')]

In [9]:
player_box_df.drop(columns=columns_to_drop, inplace=True)

In [10]:
player_box_df.shape

(201805, 108)

Dropping the rank columns removes 80 columns from the dataframe. Next, I check for null values.

In [11]:
columns_with_null_values = player_box_df.columns[player_box_df.isna().any()].tolist()
columns_with_null_values

['FG_PCT', 'FG3_PCT', 'FT_PCT', 'AVAILABLE_FLAG']

In [12]:
player_box_df[pd.isna(player_box_df['AVAILABLE_FLAG'])].head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PCT_FGM,PCT_FTA,PCT_FTM,PCT_OREB,PCT_PF,PCT_PFD,PCT_PTS,PCT_REB,PCT_STL,PCT_TOV
20105,22015,201588,George Hill,1610612754,IND,Indiana Pacers,21500276,2015-12-02,IND @ LAC,W,...,0.000,0.194,0.192,0.0,0.143,0.280,0.070,0.114,0.000,0.231
20106,22015,201941,Jordan Hill,1610612754,IND,Indiana Pacers,21500276,2015-12-02,IND @ LAC,W,...,0.308,0.300,0.278,0.4,0.333,0.250,0.277,0.364,0.000,0.200
20150,22015,101145,Monta Ellis,1610612754,IND,Indiana Pacers,21500276,2015-12-02,IND @ LAC,W,...,0.182,0.061,0.077,0.0,0.063,0.038,0.135,0.118,0.286,0.100
20151,22015,201155,Rodney Stuckey,1610612754,IND,Indiana Pacers,21500276,2015-12-02,IND @ LAC,W,...,0.250,0.273,0.316,0.0,0.143,0.200,0.250,0.179,0.250,0.000
20152,22015,203143,Pablo Prigioni,1610612746,LAC,LA Clippers,21500276,2015-12-02,LAC vs. IND,L,...,0.111,0.333,0.400,0.0,0.167,0.333,0.167,0.100,0.500,0.000


In [13]:
player_box_df['AVAILABLE_FLAG'].isna().sum()

41

The available_flag column has many null values and will not help predict points, so I get rid of it.

In [14]:
player_box_df.drop(['AVAILABLE_FLAG'],axis=1,inplace=True)

In [15]:
player_box_df.shape

(201805, 107)

In [16]:
player_box_df[pd.isna(player_box_df['FG_PCT'])].head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,PCT_FGM,PCT_FTA,PCT_FTM,PCT_OREB,PCT_PF,PCT_PFD,PCT_PTS,PCT_REB,PCT_STL,PCT_TOV
2,22015,201166,Aaron Brooks,1610612741,CHI,Chicago Bulls,21501222,2016-04-13,CHI vs. PHI,W,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.000,0.0,0.0
83,22015,2365,Chris Andersen,1610612763,MEM,Memphis Grizzlies,21501227,2016-04-13,MEM @ GSW,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143,0.0,0.0
129,22015,202684,Tristan Thompson,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000,0.0,0.0
164,22015,203527,Ryan Kelly,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
237,22015,203939,Dwight Powell,1610612742,DAL,Dallas Mavericks,21501223,2016-04-13,DAL vs. SAS,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,1.0,0.0


For the three percentage-based columns, if a player attempts 0 shots in that shot category, the percentage is stored as NaN. I replace these Nan values with 0 because the player scored 0 points off of shots from those distances, even if they didn't attempt to because they took 0 shots.

In [17]:
player_box_df.fillna(0,inplace=True)

In [18]:
columns_with_null_values = player_box_df.columns[player_box_df.isna().any()].tolist()
columns_with_null_values

[]

There are a few other features that do not provide information about player performance, so I drop those as well.

In [19]:
columns_to_drop = ['FANTASY_PTS','VIDEO_AVAILABLE','NICKNAME','NBA_FANTASY_PTS']

In [20]:
player_box_df.drop(columns=columns_to_drop, inplace=True)

In [21]:
player_box_df.shape

(201805, 103)

The player box score data has no more null values, and I have gotten rid of irrelevant columns. Now it's time for team box scores.

## Cleaning Team Box Score Data

In [22]:
team_box_df = pd.read_csv('./Data/team_box_scores.csv')

In [23]:
team_box_df.shape

(19038, 148)

In [24]:
team_box_df.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,PCT_PTS_OFF_TOV,PCT_PTS_OFF_TOV_RANK,PCT_PTS_PAINT,PCT_PTS_PAINT_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,2015-16,1610612762,UTA,Utah Jazz,21501228,2016-04-13T00:00:00,UTA @ LAL,L,48.0,39,...,0.135,1546,0.563,127,0.367,2102,0.111,1405,0.308,2109
1,2015-16,1610612754,IND,Indiana Pacers,21501225,2016-04-13T00:00:00,IND @ MIL,W,48.0,39,...,0.165,1074,0.536,210,0.393,1956,0.000,1715,0.282,2230
2,2015-16,1610612737,ATL,Atlanta Hawks,21501221,2016-04-13T00:00:00,ATL @ WAS,L,48.0,32,...,0.133,1603,0.327,2131,0.429,1703,0.091,1572,0.313,2088
3,2015-16,1610612759,SAS,San Antonio Spurs,21501223,2016-04-13T00:00:00,SAS @ DAL,W,48.0,35,...,0.146,1376,0.438,1001,0.379,2037,0.000,1715,0.314,2078
4,2015-16,1610612746,LAC,LA Clippers,21501229,2016-04-13T00:00:00,LAC @ PHX,L,48.0,43,...,0.105,2016,0.438,990,0.588,487,0.000,1715,0.465,763


Just like with the player box scores, the team box scores have over 100 columns. I use a list to look at all of the columns.

In [25]:
column_list = team_box_df.columns.tolist()
column_list

['SEASON_YEAR',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'FGM_RANK',
 'FGA_RANK',
 'FG_PCT_RANK',
 'FG3M_RANK',
 'FG3A_RANK',
 'FG3_PCT_RANK',
 'FTM_RANK',
 'FTA_RANK',
 'FT_PCT_RANK',
 'OREB_RANK',
 'DREB_RANK',
 'REB_RANK',
 'AST_RANK',
 'TOV_RANK',
 'STL_RANK',
 'BLK_RANK',
 'BLKA_RANK',
 'PF_RANK',
 'PFD_RANK',
 'PTS_RANK',
 'PLUS_MINUS_RANK',
 'AVAILABLE_FLAG',
 'TEAM_GAME_ID',
 'AST_PCT',
 'AST_PCT_RANK',
 'AST_RATIO',
 'AST_RATIO_RANK',
 'AST_TO',
 'AST_TO_RANK',
 'DEF_RATING',
 'DEF_RATING_RANK',
 'DREB_PCT',
 'DREB_PCT_RANK',
 'EFG_PCT',
 'EFG_PCT_RANK',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'NET_RATING',
 'NET_RATING_RANK',
 'OFF_RATING',
 'OFF_

Just like with the player box scores, the team box scores also contain several rank columns to drop. Available flag is present here as well, and I drop that. I don't see any other categories that need to get dropped, so the next step will be checking for NaN values. Later, I am going to have to change the column names because some of them are the same as the player columns.

In [26]:
columns_to_drop = [col for col in team_box_df.columns if col.endswith('RANK')]
team_box_df.drop(columns=columns_to_drop, inplace=True)

In [27]:
team_box_df.shape

(19038, 80)

In [28]:
team_box_df.drop(columns='AVAILABLE_FLAG',inplace=True)

In [29]:
team_box_df.shape

(19038, 79)

In [30]:
columns_with_null_values = team_box_df.columns[team_box_df.isna().any()].tolist()
columns_with_null_values

[]

Excellent! There are no null values. I do need to label the columns so that they are clear in relation to the player box score data columns, though.

In [31]:
team_box_df.columns = team_box_df.columns[:7].tolist() + ['TEAM_' + column for column in team_box_df.columns[7:]]

In [32]:
team_box_df.columns

Index(['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'TEAM_WL', 'TEAM_MIN', 'TEAM_FGM', 'TEAM_FGA',
       'TEAM_FG_PCT', 'TEAM_FG3M', 'TEAM_FG3A', 'TEAM_FG3_PCT', 'TEAM_FTM',
       'TEAM_FTA', 'TEAM_FT_PCT', 'TEAM_OREB', 'TEAM_DREB', 'TEAM_REB',
       'TEAM_AST', 'TEAM_TOV', 'TEAM_STL', 'TEAM_BLK', 'TEAM_BLKA', 'TEAM_PF',
       'TEAM_PFD', 'TEAM_PTS', 'TEAM_PLUS_MINUS', 'TEAM_TEAM_GAME_ID',
       'TEAM_AST_PCT', 'TEAM_AST_RATIO', 'TEAM_AST_TO', 'TEAM_DEF_RATING',
       'TEAM_DREB_PCT', 'TEAM_EFG_PCT', 'TEAM_E_DEF_RATING',
       'TEAM_E_NET_RATING', 'TEAM_E_OFF_RATING', 'TEAM_E_PACE',
       'TEAM_NET_RATING', 'TEAM_OFF_RATING', 'TEAM_OREB_PCT', 'TEAM_PACE',
       'TEAM_PACE_PER40', 'TEAM_PIE', 'TEAM_POSS', 'TEAM_REB_PCT',
       'TEAM_TM_TOV_PCT', 'TEAM_TS_PCT', 'TEAM_FTA_RATE', 'TEAM_OPP_EFG_PCT',
       'TEAM_OPP_FTA_RATE', 'TEAM_OPP_OREB_PCT', 'TEAM_OPP_TOV_PCT',
       'TEAM_OPP_PTS_2ND_CHANCE', 'TEAM_OPP_PTS_FB', 'TEA

In [33]:
team_box_df['TEAM_GAME_ID'] = team_box_df['TEAM_TEAM_GAME_ID'].astype(str)
team_box_df.drop(columns='TEAM_TEAM_GAME_ID',inplace=True)

In [34]:
team_box_df['TEAM_GAME_ID'].sort_values()

2457     16106127370021500001
2425     16106127370021500019
2415     16106127370021500026
2377     16106127370021500039
2343     16106127370021500055
                 ...         
16727    16106127660022201148
16702    16106127660022201163
16679    16106127660022201176
16618    16106127660022201202
16599    16106127660022201218
Name: TEAM_GAME_ID, Length: 19038, dtype: object

There is a double zero in the 11 and 12 digits place in all of these IDs. The same pattern is not present in the player_box_df. I address the discrepancy to merge them.

In [35]:
team_box_df['TEAM_GAME_ID'] = team_box_df['TEAM_GAME_ID'].str[:10] + team_box_df['TEAM_GAME_ID'].str[12:]

## Cleaning Player Index Bios

In [36]:
player_bio_df = pd.read_csv('./Data/player_bios.csv')

In [37]:
player_bio_df.shape

(4245, 24)

In [38]:
player_bio_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT,SEASON_ID
0,201166,Aaron Brooks,1610612741,CHI,31.0,6-0,72,161,Oregon,USA,...,7.1,1.5,2.6,0.0,0.018,0.068,0.225,0.494,0.265,2015-16
1,203932,Aaron Gordon,1610612753,ORL,20.0,6-9,81,220,Arizona,USA,...,9.2,6.5,1.6,-1.4,0.080,0.189,0.169,0.541,0.103,2015-16
2,1626151,Aaron Harrison,1610612766,CHA,21.0,6-6,78,210,Kentucky,USA,...,0.9,0.7,0.1,-3.4,0.045,0.112,0.132,0.371,0.033,2015-16
3,203940,Adreian Payne,1610612750,MIN,25.0,6-10,82,237,Michigan State,USA,...,2.5,2.1,0.6,-12.0,0.044,0.198,0.175,0.422,0.093,2015-16
4,201143,Al Horford,1610612737,ATL,30.0,6-10,82,245,Florida,Dominican Republic,...,15.2,7.3,3.2,4.2,0.056,0.158,0.202,0.565,0.165,2015-16


In [39]:
column_list = player_bio_df.columns.tolist()
column_list

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'PLAYER_HEIGHT',
 'PLAYER_HEIGHT_INCHES',
 'PLAYER_WEIGHT',
 'COLLEGE',
 'COUNTRY',
 'DRAFT_YEAR',
 'DRAFT_ROUND',
 'DRAFT_NUMBER',
 'GP',
 'PTS',
 'REB',
 'AST',
 'NET_RATING',
 'OREB_PCT',
 'DREB_PCT',
 'USG_PCT',
 'TS_PCT',
 'AST_PCT',
 'SEASON_ID']

In [40]:
player_bio_df[['PLAYER_HEIGHT','PLAYER_HEIGHT_INCHES']]

,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES
0,6-0,72
1,6-9,81
2,6-6,78
3,6-10,82
4,6-10,82
...,...,...
4240,6-11,83
4241,6-5,77
4242,6-9,81
4243,6-9,81


In [41]:
columns_with_null_values = player_bio_df.columns[player_bio_df.isna().any()].tolist()
columns_with_null_values

['COLLEGE', 'DRAFT_ROUND', 'DRAFT_NUMBER']

While it would be somewhat interesting to explore the predictive power of colleges on points scored, it is not necessary for my project, so I am going to drop it. I am also going to drop all of the columns with season averages, since a model would not have access to full season averages if it were making predictions in real time. I drop player height as well because of its redundancy with the height in inches columns. For the missing draft round and draft number, I replace those with zero to represent players who were undrafted.

In [42]:
cols_to_drop = ['COLLEGE', 'GP','PTS', 'REB', 'AST', 'NET_RATING', 'OREB_PCT', 
                'DREB_PCT', 'USG_PCT', 'TS_PCT', 'AST_PCT', 'PLAYER_HEIGHT']

In [43]:
player_bio_df.drop(cols_to_drop,axis=1,inplace=True)

In [44]:
player_bio_df.shape

(4245, 12)

In [45]:
player_bio_df[player_bio_df.isna().any(axis=1)]

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,SEASON_ID
425,204456,T.J. McConnell,1610612755,PHI,24.0,74,200,USA,2015,0,NaN,2015-16
472,204098,Xavier Munford,1610612763,MEM,24.0,75,180,USA,2014,NaN,NaN,2015-16
547,1626184,Chasson Randle,1610612752,NYK,24.0,74,185,USA,2015,NaN,NaN,2016-17
651,1627819,Isaiah Taylor,1610612745,HOU,22.0,75,170,USA,2016,NaN,NaN,2016-17
894,1627815,Sheldon Mac,1610612764,WAS,24.0,78,200,USA,2016,NaN,NaN,2016-17
...,...,...,...,...,...,...,...,...,...,...,...,...
3631,204456,T.J. McConnell,1610612754,IND,30.0,73,190,USA,2015,0,NaN,2021-22
3687,1627782,Wayne Selden,1610612752,NYK,27.0,76,232,USA,2016,NaN,NaN,2021-22
3882,1629312,Haywood Highsmith,1610612748,MIA,26.0,77,220,USA,2018,NaN,NaN,2022-23
4041,1629623,Lindell Wigginton,1610612749,MIL,25.0,73,189,Canada,2019,NaN,NaN,2022-23


As expected, the NaNs were for players who went undrafted. I replace the NaN values with zeroes to represent this numerically.

In [46]:
player_bio_df[['DRAFT_ROUND', 'DRAFT_NUMBER']].value_counts()

DRAFT_ROUND  DRAFT_NUMBER
Undrafted    Undrafted       921
1            9               107
             5               106
             3               103
             1               103
                            ... 
2            57               13
             54               13
             53               13
             59                6
             30                3
Name: count, Length: 63, dtype: int64

The problem isn't just that there are null values. It's also that there is a string called undrafted. I am going to set that to 0 as well as both of them indicate undrafted players.

In [47]:
player_bio_df['DRAFT_ROUND'].fillna(0,inplace=True)

In [48]:
player_bio_df['DRAFT_ROUND'] = player_bio_df['DRAFT_ROUND'].replace('Undrafted',0)

In [49]:
player_bio_df['DRAFT_NUMBER'].fillna(0,inplace=True)

In [50]:
player_bio_df['DRAFT_NUMBER'] = player_bio_df['DRAFT_NUMBER'].replace('Undrafted',0)

In [51]:
player_bio_df[['DRAFT_ROUND', 'DRAFT_NUMBER']].value_counts()

DRAFT_ROUND  DRAFT_NUMBER
0            0               999
1            9               107
             5               106
             1               103
             3               103
                            ... 
2            53               13
             57               13
0            0                 9
2            59                6
             30                3
Name: count, Length: 64, dtype: int64

In [52]:
player_bio_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,SEASON_ID
0,201166,Aaron Brooks,1610612741,CHI,31.0,72,161,USA,2007,1,26,2015-16
1,203932,Aaron Gordon,1610612753,ORL,20.0,81,220,USA,2014,1,4,2015-16
2,1626151,Aaron Harrison,1610612766,CHA,21.0,78,210,USA,Undrafted,0,0,2015-16
3,203940,Adreian Payne,1610612750,MIN,25.0,82,237,USA,2014,1,15,2015-16
4,201143,Al Horford,1610612737,ATL,30.0,82,245,Dominican Republic,2007,1,3,2015-16


Draft Year has some Undrafted strings as well.

In [53]:
player_bio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4245 entries, 0 to 4244
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PLAYER_ID             4245 non-null   int64  
 1   PLAYER_NAME           4245 non-null   object 
 2   TEAM_ID               4245 non-null   int64  
 3   TEAM_ABBREVIATION     4245 non-null   object 
 4   AGE                   4245 non-null   float64
 5   PLAYER_HEIGHT_INCHES  4245 non-null   int64  
 6   PLAYER_WEIGHT         4245 non-null   int64  
 7   COUNTRY               4245 non-null   object 
 8   DRAFT_YEAR            4245 non-null   object 
 9   DRAFT_ROUND           4245 non-null   object 
 10  DRAFT_NUMBER          4245 non-null   object 
 11  SEASON_ID             4245 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 398.1+ KB


In [54]:
player_bio_df['DRAFT_YEAR'] = player_bio_df['DRAFT_YEAR'].replace('Undrafted',0)

In [55]:
player_bio_df[['DRAFT_YEAR','DRAFT_ROUND','DRAFT_NUMBER']] = player_bio_df[['DRAFT_YEAR','DRAFT_ROUND','DRAFT_NUMBER']].astype(int)

In [56]:
player_bio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4245 entries, 0 to 4244
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PLAYER_ID             4245 non-null   int64  
 1   PLAYER_NAME           4245 non-null   object 
 2   TEAM_ID               4245 non-null   int64  
 3   TEAM_ABBREVIATION     4245 non-null   object 
 4   AGE                   4245 non-null   float64
 5   PLAYER_HEIGHT_INCHES  4245 non-null   int64  
 6   PLAYER_WEIGHT         4245 non-null   int64  
 7   COUNTRY               4245 non-null   object 
 8   DRAFT_YEAR            4245 non-null   int32  
 9   DRAFT_ROUND           4245 non-null   int32  
 10  DRAFT_NUMBER          4245 non-null   int32  
 11  SEASON_ID             4245 non-null   object 
dtypes: float64(1), int32(3), int64(4), object(4)
memory usage: 348.3+ KB


In [57]:
columns_with_null_values = player_bio_df.columns[player_bio_df.isna().any()].tolist()
columns_with_null_values

[]

In [58]:
player_bio_df.head(10)

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,SEASON_ID
0,201166,Aaron Brooks,1610612741,CHI,31.0,72,161,USA,2007,1,26,2015-16
1,203932,Aaron Gordon,1610612753,ORL,20.0,81,220,USA,2014,1,4,2015-16
2,1626151,Aaron Harrison,1610612766,CHA,21.0,78,210,USA,0,0,0,2015-16
3,203940,Adreian Payne,1610612750,MIN,25.0,82,237,USA,2014,1,15,2015-16
4,201143,Al Horford,1610612737,ATL,30.0,82,245,Dominican Republic,2007,1,3,2015-16
5,2744,Al Jefferson,1610612766,CHA,31.0,82,289,USA,2004,1,15,2015-16
6,202329,Al-Farouq Aminu,1610612757,POR,25.0,81,215,USA,2010,1,8,2015-16
7,101187,Alan Anderson,1610612764,WAS,33.0,78,220,USA,0,0,0,2015-16
8,1626210,Alan Williams,1610612756,PHX,23.0,80,260,USA,0,0,0,2015-16
9,202692,Alec Burks,1610612762,UTA,24.0,78,214,USA,2011,1,12,2015-16


Everything looks good. I am going to create a new id that concatenates player and season id, labels the bio columns as such and then move on to merging the dataframes together.

In [59]:
player_bio_df['PLAYER_SEASON_ID'] = player_bio_df['PLAYER_ID'].astype(str) + player_bio_df['SEASON_ID'].astype(str)

In [60]:
player_bio_df.columns = player_bio_df.columns[:4].tolist() + ['BIO_' + column for column in player_bio_df.columns[4:]]

In [61]:
player_bio_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'BIO_AGE',
       'BIO_PLAYER_HEIGHT_INCHES', 'BIO_PLAYER_WEIGHT', 'BIO_COUNTRY',
       'BIO_DRAFT_YEAR', 'BIO_DRAFT_ROUND', 'BIO_DRAFT_NUMBER',
       'BIO_SEASON_ID', 'BIO_PLAYER_SEASON_ID'],
      dtype='object')

## Merging Data into a Single Dataframe

First, I add a team game ID column to the player box score dataframe so that the merge can occur.

In [62]:
player_box_df['TEAM_GAME_ID'] = player_box_df['TEAM_ID'].astype(str) + player_box_df['GAME_ID'].astype(str)

In [64]:
player_box_df['SEASON_YEAR']

0         2015-16
1         2015-16
2         2015-16
3         2015-16
4         2015-16
           ...   
201800    2022-23
201801    2022-23
201802    2022-23
201803    2022-23
201804    2022-23
Name: SEASON_YEAR, Length: 201805, dtype: object

In [65]:
player_box_df['PLAYER_SEASON_ID'] = player_box_df['PLAYER_ID'].astype(str) +  player_box_df['SEASON_YEAR'].astype(str)

In [66]:
player_box_df.shape

(201805, 105)

In [71]:
cols = player_box_df.columns.tolist()
cols

['SEASON_ID',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'PLAYER_GAME_ID',
 'AST_PCT',
 'AST_RATIO',
 'AST_TO',
 'DEF_RATING',
 'DREB_PCT',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_TOV_PCT',
 'E_USG_PCT',
 'FGA_PG',
 'FGM_PG',
 'NET_RATING',
 'OFF_RATING',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PIE',
 'POSS',
 'REB_PCT',
 'SEASON_YEAR',
 'TM_TOV_PCT',
 'TS_PCT',
 'USG_PCT',
 'sp_work_DEF_RATING',
 'sp_work_NET_RATING',
 'sp_work_OFF_RATING',
 'sp_work_PACE',
 'BLKA',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'PFD',
 'PTS_2ND_CHANCE',
 'PTS_FB',
 'PTS_OFF_TOV',
 'PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_AST_3PM',
 'PCT_AST_FGM',
 'PCT_FGA_2PT',
 'PCT_FGA_3PT',
 'PCT_

First, I merge together the player box score df and the team box score df.

In [68]:
merged_df = pd.merge(player_box_df,team_box_df,on='TEAM_GAME_ID',how='left')

In [69]:
merged_df.shape

(201805, 183)

Next, I merge together the merged df with the player bio df.

In [76]:
merged_df = pd.merge(merged_df,player_bio_df,left_on='PLAYER_SEASON_ID',right_on='BIO_PLAYER_SEASON_ID',how='left')

In [77]:
columns_with_null_values = merged_df.columns[merged_df.isna().any()].tolist()
columns_with_null_values

[]

In [78]:
col_list = merged_df.columns.tolist()
col_list

['SEASON_ID',
 'PLAYER_ID_x',
 'PLAYER_NAME_x',
 'TEAM_ID_x',
 'TEAM_ABBREVIATION_x',
 'TEAM_NAME_x',
 'GAME_ID_x',
 'GAME_DATE_x',
 'MATCHUP_x',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'PLAYER_GAME_ID',
 'AST_PCT',
 'AST_RATIO',
 'AST_TO',
 'DEF_RATING',
 'DREB_PCT',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_TOV_PCT',
 'E_USG_PCT',
 'FGA_PG',
 'FGM_PG',
 'NET_RATING',
 'OFF_RATING',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PIE',
 'POSS',
 'REB_PCT',
 'SEASON_YEAR_x',
 'TM_TOV_PCT',
 'TS_PCT',
 'USG_PCT',
 'sp_work_DEF_RATING',
 'sp_work_NET_RATING',
 'sp_work_OFF_RATING',
 'sp_work_PACE',
 'BLKA',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'PFD',
 'PTS_2ND_CHANCE',
 'PTS_FB',
 'PTS_OFF_TOV',
 'PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_AST_3PM',
 'PCT_AST_FGM',
 'PCT_FGA_2PT',
 'PC

A few columns got duplicated in the merge. I identify and remove them.

In [79]:
duplicates = [col for col in merged_df.columns if col.endswith('y')]
duplicates

['SEASON_YEAR_y',
 'TEAM_ID_y',
 'TEAM_ABBREVIATION_y',
 'TEAM_NAME_y',
 'GAME_ID_y',
 'GAME_DATE_y',
 'MATCHUP_y',
 'PLAYER_ID_y',
 'PLAYER_NAME_y']

In [80]:
merged_df.drop(columns = duplicates,inplace=True)

In [81]:
merged_df.shape

(201805, 187)

Now that the box score data is cleaned and merged, I export it to prepare for EDA and Modeling.

In [82]:
merged_df.to_csv('./Data/cleaned_and_merged_data.csv',index=False)